# Description

It projects input data into a UMAP representation.

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from IPython.display import display

import pandas as pd
import umap
import matplotlib.pyplot as plt
import seaborn as sns

import conf
from utils import generate_result_set_name

# Settings

In [3]:
INPUT_FILEPATH = Path(
    conf.RESULTS['PROJECTIONS_DIR'],
    'projection-smultixcan-efo_partial-mashr-zscores.pkl'
).resolve()
display(INPUT_FILEPATH)

input_filepath_stem = INPUT_FILEPATH.stem
display(input_filepath_stem)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/projections/projection-smultixcan-efo_partial-mashr-zscores.pkl')

'projection-smultixcan-efo_partial-mashr-zscores'

In [4]:
# number of components to use in the dimensionality reduction step
DR_OPTIONS = {
    'n_components': [5, 10, 20, 30, 40, 50],
    'metric': 'euclidean',
    'n_neighbors': 15,
    'random_state': 0,
}

In [5]:
# output dir for this notebook
RESULTS_DIR = Path(
    conf.RESULTS["DATA_TRANSFORMATIONS_DIR"],
    'umap'
).resolve()
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

display(RESULTS_DIR)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/data_transformations/umap')

In [6]:
# dictionary containing all options/settings (used to generate filenames)
ALL_OPTIONS = DR_OPTIONS.copy()
# ALL_OPTIONS['proj_percentile'] = PERCENTILE_NAME

display(ALL_OPTIONS)

{'n_components': [5, 10, 20, 30, 40, 50],
 'metric': 'euclidean',
 'n_neighbors': 15,
 'random_state': 0}

# Load input file

In [7]:
data = pd.read_pickle(INPUT_FILEPATH).T

In [8]:
data.shape

(3749, 987)

In [9]:
data.head()

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,...,LV978,LV979,LV980,LV981,LV982,LV983,LV984,LV985,LV986,LV987
100001_raw-Food_weight,-0.018452,0.052938,-0.003629,0.028359,-0.015500,0.035854,-0.015439,0.023007,0.017368,0.026811,...,0.033560,0.047126,-0.036183,0.068750,0.023462,-0.030111,0.011272,-0.017171,0.016078,-0.022283
100002_raw-Energy,-0.043782,-0.012041,-0.011772,-0.006148,0.007011,0.018142,0.003144,0.018049,0.006926,0.038587,...,0.004833,0.022842,-0.009519,-0.000258,0.059764,-0.028394,-0.005967,0.045269,-0.007684,-0.018910
100003_raw-Protein,-0.021514,-0.028537,0.009441,0.007808,0.012707,0.021681,-0.006315,0.016129,0.000076,-0.001702,...,0.029704,0.029135,-0.056508,-0.002032,0.001189,-0.025507,-0.013012,0.037458,-0.009592,-0.016718
100004_raw-Fat,-0.030454,-0.052542,0.000459,-0.039613,0.006191,0.029523,0.000747,0.011876,-0.025758,0.025099,...,0.015900,0.016482,0.007409,-0.006833,0.036457,-0.034531,0.015365,0.023796,-0.017477,-0.005397
100005_raw-Carbohydrate,-0.017428,0.003757,-0.003708,-0.000929,-0.000647,-0.005729,0.024970,0.011531,0.035043,0.025159,...,-0.010071,0.002266,0.006664,0.007380,0.029940,-0.006989,0.014807,0.050208,0.005352,-0.049218


# UMAP

In [10]:
def get_umap_proj(orig_data, options):
    umap_obj = umap.UMAP(**{k:v for k, v in options.items() if k in DR_OPTIONS})
    umap_obj = umap_obj.fit(orig_data)
    umap_data = umap_obj.transform(orig_data)
    return pd.DataFrame(
        data=umap_data,
        index=orig_data.index.copy(),
        columns=[f'UMAP{i+1}' for i in range(umap_data.shape[1])]
    )

In [11]:
# for n_comp, n_neigh in product(DR_OPTIONS['n_components'], DR_OPTIONS['n_neighbors']):
for n_comp in DR_OPTIONS['n_components']:
    print(f'# components: {n_comp}')
    
    options = ALL_OPTIONS.copy()
    options['n_components'] = n_comp
    
    dr_data = get_umap_proj(data, options)
    
    display(dr_data.shape)
    assert dr_data.shape == (data.shape[0], n_comp)
    
    display(dr_data.iloc[:, 0:5].describe())
    
    # save
    output_file = Path(
        RESULTS_DIR,
        generate_result_set_name(
            options,
            prefix=f'umap-{input_filepath_stem}-',
            suffix='.pkl'
        )
    ).resolve()
    display(output_file)
    
    dr_data.to_pickle(output_file)
    
    print('\n')

# components: 5


(3749, 5)

,UMAP1,UMAP2,UMAP3,UMAP4,UMAP5
count,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000
mean,3.996364,-4.579150,-2.381185,12.718588,15.357991
std,0.828690,1.125089,0.734250,1.221331,0.908778
min,2.210563,-5.985320,-4.135514,5.086887,8.194320
25%,3.442145,-5.132524,-2.808691,12.422697,15.048211
50%,3.875299,-4.676681,-2.372222,12.952255,15.438498
75%,4.388280,-4.222258,-2.054847,13.364546,15.816606
max,8.131745,12.219553,4.790262,14.530914,17.804277


PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/data_transformations/umap/umap-projection-smultixcan-efo_partial-mashr-zscores-metric_euclidean-n_components_5-n_neighbors_15-random_state_0.pkl')



# components: 10


...

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/data_transformations/umap/umap-projection-smultixcan-efo_partial-mashr-zscores-metric_euclidean-n_components_40-n_neighbors_15-random_state_0.pkl')



# components: 50


(3749, 50)

,UMAP1,UMAP2,UMAP3,UMAP4,UMAP5
count,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000
mean,12.190821,9.693271,8.821040,5.151597,13.368401
std,0.574257,0.511677,0.297742,0.204645,0.511793
min,7.259162,-0.741623,4.879749,4.516759,8.868875
25%,12.116760,9.599879,8.706084,5.007544,13.277666
50%,12.264053,9.706082,8.825241,5.154793,13.428452
75%,12.401225,9.816405,8.944929,5.299308,13.564143
max,13.152287,10.181146,10.434216,5.804502,14.315068


PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/data_transformations/umap/umap-projection-smultixcan-efo_partial-mashr-zscores-metric_euclidean-n_components_50-n_neighbors_15-random_state_0.pkl')